### Set up Pyro connection to the board
``soc`` and ``soccfg`` live on the board, and we interact with them through pyro proxy objects.  This section of code connects to the pyro server running on the board to receive these objects

In [1]:
## ------------------------------------------------ ##
import Pyro4
from qick import *

Pyro4.config.SERIALIZER = "pickle"
Pyro4.config.PICKLE_PROTOCOL_VERSION=4

ns_host = "192.168.137.93"
ns_port = 8883
proxy_name = "DMQIS_qick"
ns = Pyro4.locateNS(host=ns_host, port=ns_port)

for k,v in ns.list().items():
    print(k,v)

soc = Pyro4.Proxy(ns.lookup(proxy_name))
soccfg = QickConfig(soc.get_cfg())
#print(soccfg)
## ------------------------------------------------ ##

Pyro.NameServer PYRO:Pyro.NameServer@192.168.137.93:8883
DMQIS_qick PYRO:obj_8a18b384aecc4e1d939860471324ce0e@192.168.137.103:42509


c:\Users\slab.DESKTOP-JFLC5N4\radioconda\envs\DMQIS_env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Basic Setup Parameters
Here we specify the channel connections between the fridge and the board.

I have set things up with the following configuration
* DAC 6: goes into warm splitter into fridge line A16
* DAC 2: goes into warm splitter into fridge line A16
* ADC 0: comes out of fridge line C7

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from qick import *
import sys
sys.path.append('./Libraries')
from qick_data import *
from qick_helpers import *
from qick_programs import *
import tqdm

datapath = 'c:\\_Data\\Hannah\\SQUAT_freq_power_scan'

### Set up data handling

In [8]:
## Initialize a measurement object
## This object holds dictionaries
## Not much else to it.  Just a container that handles export/import
meas = QICKdata()

## Next we fill the metadata dictionary
meas.set_metadata(device="C11", qubits=["1"], meas_type="freq_power_scan_CB")

## Then specify the connection channels
meas.set_hw_cfg(res_ch=chargebias_ch, qu_ch=None, ro_ch=ro_ch)

## Finally specify pulse parameters
meas.meas_cfg = {"relax_delay_us" : 0,
            "adc_trig_offset_us"  : 0.506,
            "readout_length_us"   : 20, #0.2,
   
            "res_style"     : "const",
            ## "res_gain"      : 1100, #30000,
            "res_length_us" : 20, #0.279,
            "res_phase"     : 0,
            ## commenting out because this is a freq scan
            ## "res_freq" : 3000, ## MHz

            "reps"          : 2000,   ## Averages in firmware
            "soft_avgs"     : 1,      ## Averages in software
       }

## Convert time variables from us to clock ticks
meas = convert_time_vars(soccfg, meas)
print(meas)

## Set up sweep values
gains = np.linspace(100, 20000, 15)
RF_freqs = np.linspace(7340, 7370, num=600)

## Do measurement
Ivals = np.zeros((len(gains), len(RF_freqs)))
Qvals = np.zeros((len(gains), len(RF_freqs)))
config = {**meas.hw_cfg, **meas.meas_cfg}
for j, g in enumerate(tqdm.tqdm(gains)):
    ## Gain needs to be an int
    g = int(g)
    gains[j] = g
    config['res_gain'] = g
    for i, f in enumerate(RF_freqs):
        config["res_freq"] = f
        prog =oneTonePulse(soccfg, config)
        [[I]], [[Q]] = prog.acquire(soc)
        Ivals[j][i] = I
        Qvals[j][i] = Q

##  IMPORTANT make sure you save the data
meas.meas_data = {"gains" : gains, "RF_freqs" : RF_freqs, "Ivals" : Ivals, "Qvals" : Qvals, "datastorage" : "IQvalue[gain index][freq index]"}
meas.write_H5(datapath)


Creating dataset with series 20240513_115743
converting time variables from us to clock ticks
====---------------------------====
         20240513_115743
====---------------------------====
       Device =  C11
       Qubits =  ['1']
    Meas Type =  freq_power_scan_CB
====---------------------------====
20240513_115743


100%|██████████| 15/15 [09:41<00:00, 38.80s/it]

Saving data as: 20240513_115743_freq_power_scan_CB.h5
